# Preparing a dataset

* How does the dataset handle invalid values? 
* What do we want to do with null values?
* Do we want to summarise, group or filter the data?

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("Diabetes.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Pregnancies                 768 non-null int64
Glucose                     768 non-null int64
BloodPressure               768 non-null int64
SkinThickness               768 non-null int64
Insulin                     768 non-null int64
BMI                         768 non-null float64
DiabetesPedigreeFunction    768 non-null float64
Age                         768 non-null int64
Outcome                     768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [2]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


So it looks like they are using `0` values where they dont have data. I don't think theyre using `NaN` at all, but if they were we could either drop those rows (`dropna`) or fill them to some value (`fillna`). Because they're using `0` already, it might be prudent to do this just in case.

In [3]:
df = df.fillna(0)

So now we know there are `NaN` values. We could also have just, you know, checked for `NaN`, but now I'm trying to show functions you can use.

So, what to do with these zero values. In some cases we could fill them with something sensible, but that normally just biases the data. So mostly we'd ignore them. So what we want to do is ask *how* we want to use the data. Will we be using `SkinThickness`? Do we care if there are non-physical outliers?

If we cared about Glucose, BMI and Age primarily, we could get rid of a ton of these issues but only looking at those columns

In [4]:
df2 = df[["Glucose", "BMI", "Age", "Outcome"]]

In [5]:
df2.head()

,Glucose,BMI,Age,Outcome
0,148,33.6,50,1
1,85,26.6,31,0
2,183,23.3,32,1
3,89,28.1,21,0
4,137,43.1,33,1


In [7]:
df2.describe()

,Glucose,BMI,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000
mean,120.894531,31.992578,33.240885,0.348958
std,31.972618,7.884160,11.760232,0.476951
min,0.000000,0.000000,21.000000,0.000000
25%,99.000000,27.300000,24.000000,0.000000
50%,117.000000,32.000000,29.000000,0.000000
75%,140.250000,36.600000,41.000000,1.000000
max,199.000000,67.100000,81.000000,1.000000


But now lets get rid of any stray zeros. 

What we want to do is find a row with *any* number of zeros and remove that row. Or in terms of applying a mask, find the rows which have any zero (True), invert that (to False) so when we apply the mask using `loc`, the False entries get dropped.

In [14]:
df3 = df2.loc[~(df2[df2.columns[:-1]] == 0).any(axis=1)]
df3.describe()
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 0 to 767
Data columns (total 4 columns):
Glucose    752 non-null int64
BMI        752 non-null float64
Age        752 non-null int64
Outcome    752 non-null int64
dtypes: float64(1), int64(3)
memory usage: 29.4 KB


Great, so we've selected the data we cared about, made sure it has no `null`-like values. We'll go on to checking things look sane with some plots in the next section. One final thing we could do is to group the data by outcome. It might make it easier to look for patterns in diagnoses.

We can do this either by splitting out the DataFrame into two (one for yes and one for no), or if we wanted summary statistics we could use the `groupBy` function:

In [16]:
df3.groupby("Outcome").mean()

,Glucose,BMI,Age
Outcome,,,
0,110.825820,30.876434,31.309426
1,142.488636,35.371970,37.015152


And what this can tell us is that, in general, the higher your glucose level, the more overweight you are, and the older you are, the greater your chance of being diagnosed with diabetes. Which, perhaps, is not that surprising.

We can do other things using the `groupby` statement, like so:

In [18]:
df3.groupby("Outcome").agg({"Glucose": "mean", "BMI": "median", "Age": "sum"})

,Glucose,BMI,Age
Outcome,,,
0,110.825820,30.10,15279
1,142.488636,34.25,9772


In [19]:
df3.groupby("Outcome").agg(["mean", "median"])

Glucose               BMI               Age       
               mean median       mean median       mean median
Outcome                                                       
0        110.825820  107.5  30.876434  30.10  31.309426     27
1        142.488636  140.5  35.371970  34.25  37.015152     36

We can also split the dataset into positive and negative outcomes if we wanted. *If*.

In [21]:
positive = df3.loc[df3["Outcome"] == 1]
negative = df3.loc[df3["Outcome"] == 0]
print(positive.shape, negative.shape)

(264, 4) (488, 4)


We won't use this splitting just yet, so lets save out the cleaned and prepared dataset, `df3` to file, so our analysis code can load it in the future without having to copy-paste the data prep code into future notebooks.

In [23]:
df3.to_csv("clean_diabetes.csv", index=False)